# considrations:

- the total_reads_percell and non_zero_peaks_percell is calculated before zscore filtering
- quantile filtering is calculated for each conditions+TF seperatly not for the mixture of conditions of each TF together ! 
- env : scanpy (version=1.10.4)


In [ ]:
#imports

import pandas as pd
import numpy as np

import seaborn as sns


import scanpy as sc
#from anndata import AnnData
import anndata


from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages  # Import PdfPages


from sklearn.cluster import KMeans

import scipy.stats as stats


In [ ]:
# open the count matrix file for cleaned peaks:
input_file = "./merged_file.txt"
merged_df = pd.read_csv(input_file, sep="\t", index_col="region")
merged_df.index.name = None

# make adata object from the merged_df so that the obs are the cells
transposed_df = merged_df.T
adata = sc.AnnData(transposed_df)

#display(adata)

# Split the observation names (index of adata.obs) by "_" for annotation of the cells
obs_names_split = adata.obs_names.str.split("_", expand=True)

# Add new columns to adata.obs
adata.obs["TR"] = obs_names_split.get_level_values(1)  # Second part (index 1)
adata.obs["condition"] = obs_names_split.get_level_values(2)  # Third part (index 2)
adata.obs["replicate"] = obs_names_split.get_level_values(3)  # Fourth part (index 3)
adata.obs['TR_Condition_initial'] = adata.obs['TR'].astype(str) + "_" + adata.obs['condition'].astype(str)

# some calculations for the cells (before filtering)
adata.obs['total_reads_percell'] = adata.X.sum(axis=1)
adata.obs['non_zero_peaks_percell'] = (adata.X != 0).sum(axis=1)

adata.obs['log1p_total_reads_percell'] = np.log1p(adata.obs['total_reads_percell'])
adata.obs['log1p_non_zero_peaks_percell'] = np.log1p(adata.obs['non_zero_peaks_percell'])

#display(adata.obs)
#print(obs_names_split)

# calculate some statistics about the counts (total read count and non zero count peaks)
adata.var['total_reads_perpeaks'] = adata.X.sum(axis=0)
adata.var['non_zero_cells_perpeaks'] = (adata.X != 0).sum(axis=0)

# Apply log1p transformation to 'peak_read_sum' and 'num_non_zero_cells'
adata.var['log1p_total_reads_perpeaks'] = np.log1p(adata.var['total_reads_perpeaks'])
adata.var['log1p_non_zero_cells_perpeaks'] = np.log1p(adata.var['non_zero_cells_perpeaks'])

#display(adata.var)

# open totalBAMreadsPerCell.txt as a dataframe
merged_df_totalreads = pd.read_csv("totalBAMreadsPerCell.txt", sep=" ", header=None, index_col=0)
merged_df_totalreads.index.name = None
# rename the column 1 to "totalReads"
merged_df_totalreads.columns = ["totalReads"]
#display(merged_df_totalreads)

# make the order of the index of merged_df_totalreads the same as the order of the adata.obs_names
merged_df_totalreads = merged_df_totalreads.reindex(adata.obs_names)
#display(merged_df_totalreads)

# add the merged_df_totalreads to the adata.obs
adata.obs["total_reads_BAM_percell"] = merged_df_totalreads["totalReads"].values

# FRiP calculation
adata.obs["FRiP"] = adata.obs['total_reads_percell']/adata.obs["total_reads_BAM_percell"]

display(adata.obs)


,TR,condition,replicate,TR_Condition_initial,total_reads_percell,non_zero_peaks_percell,log1p_total_reads_percell,log1p_non_zero_peaks_percell,total_reads_BAM_percell,FRiP
R15C43_NANOG_EpiLC_1_S826,NANOG,EpiLC,1,NANOG_EpiLC,298,13,5.700444,2.639057,606,0.491749
R19C52_OCT4_ESC_2_S2026,OCT4,ESC,2,OCT4_ESC,0,0,0.000000,0.000000,4,0.000000
R33C71_YAP1_ESC_1_S2618,YAP1,ESC,1,YAP1_ESC,0,0,0.000000,0.000000,0,NaN
R0C10_MYC_ESC_1_S386,MYC,ESC,1,MYC_ESC,6423,86,8.767796,4.465908,15046,0.426891
R65C23_YAP1_EpiLC_1_S2294,YAP1,EpiLC,1,YAP1_EpiLC,206,4,5.332719,1.609438,592,0.347973
...,...,...,...,...,...,...,...,...,...,...
R68C42_YAP1_EpiLC_2_S2512,YAP1,EpiLC,2,YAP1_EpiLC,591,7,6.383507,2.079442,2704,0.218565
R39C1_MYC_EpiLC_2_S317,MYC,EpiLC,2,MYC_EpiLC,23112,147,10.048151,4.997212,59418,0.388973
R61C25_OCT4_EpiLC_2_S1814,OCT4,EpiLC,2,OCT4_EpiLC,2826,43,7.946971,3.784190,8392,0.336749
R5C35_MYC_ESC_1_S446,MYC,ESC,1,MYC_ESC,2754,30,7.921173,3.433987,6442,0.427507


In [3]:
#count how many cells have NaN in the FRiP column
print(adata.obs['FRiP'].isna().sum())

print(adata.shape)

# Filter out cells with NaN in the FRiP column
adata = adata[~adata.obs['FRiP'].isna(), :]
print(adata.shape)

68
(2892, 24897)
(2824, 24897)


In [5]:
#to see the size of adata without 'YAP1'
adata[adata.obs['TR']!='YAP1']

View of AnnData object with n_obs × n_vars = 2117 × 24897
    obs: 'TR', 'condition', 'replicate', 'TR_Condition_initial', 'total_reads_percell', 'non_zero_peaks_percell', 'log1p_total_reads_percell', 'log1p_non_zero_peaks_percell', 'total_reads_BAM_percell', 'FRiP'
    var: 'total_reads_perpeaks', 'non_zero_cells_perpeaks', 'log1p_total_reads_perpeaks', 'log1p_non_zero_cells_perpeaks'

In [12]:
# plot the initial data (all clean peaks before z-score and quantile filtering)

# Violin Plot with combined group
violin_pdf_filename = "VP_log1pNonZeroPeaksPerCell_noZscore_noQuantileFiltered.pdf"
with PdfPages(violin_pdf_filename) as pdf:
    plt.figure(figsize=(12,6))
    sc.pl.violin(
        adata[adata.obs['TR']!='YAP1'],
        keys="log1p_non_zero_peaks_percell",
        groupby="TR_Condition_initial",  # Use the new combined column
        rotation=90 , 
        show=False
    )  
    plt.xticks(rotation=90)
    plt.xticks(fontsize=8) # Reduce font size
    plt.yticks(fontsize=8) # Reduce font size
    plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels
    plt.title("log1p_non_zero_peaks_percell_noZscore_noQuantileFiltered")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

# Violin Plot with combined group
violin_pdf_filename = "VP_log1pTotalReadsPercell_noZscore_noQuantileFiltered.pdf"
with PdfPages(violin_pdf_filename) as pdf:
    plt.figure(figsize=(12,6))
    sc.pl.violin(
        adata[adata.obs['TR']!='YAP1'],
        keys="log1p_total_reads_percell",
        groupby="TR_Condition_initial",  # Use the new combined column
        rotation=90 , 
        show=False
    )  
    plt.xticks(rotation=90)
    plt.xticks(fontsize=8) # Reduce font size
    plt.yticks(fontsize=8) # Reduce font size
    plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels
    plt.title("log1p_total_reads_percell_noZscore_noQuantileFiltered")
    plt.tight_layout()
    pdf.savefig()
    plt.close()



/Users/nadia/anaconda3/envs/scanpy/lib/python3.10/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/nadia/anaconda3/envs/scanpy/lib/python3.10/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/nadia/anaconda3/envs/scanpy/lib/python3.10/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/Users/nadia/anaconda3/envs/scanpy/lib/python3.10/site-packages/anndata/_core/anndata.py:1146: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  df[key] = c
/var/folders/yv/0k5783m54kz5cltf2_077mrm0000gn/T/ipykernel_69932/2106904533.py:19: UserWarning: This figure includes Axes that are not compatible with tight_lay

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

# separate the TF and conditions into 8 seperate adata


In [ ]:
# read the TFs sig peaks files and get the list of the sig peaks for each TF

# open the zscored filtered peaks IDs
MYC_sig_peaks = pd.read_csv("MYC_zscores_pvals_sig_REVIEW_NORMALIZED.txt", sep="\t", index_col="region")
NANOG_sig_peaks = pd.read_csv("NANOG_zscores_pvals_sig_REVIEW_NORMALIZED.txt", sep="\t", index_col="region")
OCT4_sig_peaks = pd.read_csv("OCT4_zscores_pvals_sig_REVIEW_NORMALIZED.txt", sep="\t", index_col="region")

MYC_sig_peaks.index.name = None
peaklist_Myc = MYC_sig_peaks.index.tolist()
print(MYC_sig_peaks.shape)

NANOG_sig_peaks.index.name = None
peaklist_Nanog = NANOG_sig_peaks.index.tolist()
print(NANOG_sig_peaks.shape)

OCT4_sig_peaks.index.name = None
peaklist_Oct4 = OCT4_sig_peaks.index.tolist()
print(OCT4_sig_peaks.shape)


(10048, 2)
(10328, 2)
(11041, 2)


In [8]:
#Filter the peack that are passed the z-score threshold for each TF
 
adata #['MYC', 'NANOG', 'OCT4', 'YAP1']

adata.layers['normalized'] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=10000, layer='normalized', inplace=True)

adata_ESC_MYC_all = adata[(adata.obs['condition']=='ESC') & (adata.obs['TR']=='MYC'),:].copy()
adata_ESC_MYC = adata_ESC_MYC_all[:, peaklist_Myc].copy()

# some calculations for the cells (after zscore filtering)
adata_ESC_MYC.obs['total_reads_percell_zscored'] = adata_ESC_MYC.X.sum(axis=1)
adata_ESC_MYC.obs['non_zero_peaks_percell_zscored'] = (adata_ESC_MYC.X != 0).sum(axis=1)

adata_ESC_MYC.obs['log1p_total_reads_percell_zscored'] = np.log1p(adata_ESC_MYC.obs['total_reads_percell_zscored'])
adata_ESC_MYC.obs['log1p_non_zero_peaks_percell_zscored'] = np.log1p(adata_ESC_MYC.obs['non_zero_peaks_percell_zscored'])


adata_ESC_NANOG_all = adata[(adata.obs['condition']=='ESC') & (adata.obs['TR']=='NANOG'),:].copy()
adata_ESC_NANOG = adata_ESC_NANOG_all[:, peaklist_Nanog].copy()

# some calculations for the cells (after zscore filtering)
adata_ESC_NANOG.obs['total_reads_percell_zscored'] = adata_ESC_NANOG.X.sum(axis=1)
adata_ESC_NANOG.obs['non_zero_peaks_percell_zscored'] = (adata_ESC_NANOG.X != 0).sum(axis=1)

adata_ESC_NANOG.obs['log1p_total_reads_percell_zscored'] = np.log1p(adata_ESC_NANOG.obs['total_reads_percell_zscored'])
adata_ESC_NANOG.obs['log1p_non_zero_peaks_percell_zscored'] = np.log1p(adata_ESC_NANOG.obs['non_zero_peaks_percell_zscored'])


adata_ESC_OCT4_all = adata[(adata.obs['condition']=='ESC') & (adata.obs['TR']=='OCT4'),:].copy()
adata_ESC_OCT4 = adata_ESC_OCT4_all[:, peaklist_Oct4].copy()

# some calculations for the cells (after zscore filtering)
adata_ESC_OCT4.obs['total_reads_percell_zscored'] = adata_ESC_OCT4.X.sum(axis=1)
adata_ESC_OCT4.obs['non_zero_peaks_percell_zscored'] = (adata_ESC_OCT4.X != 0).sum(axis=1)

adata_ESC_OCT4.obs['log1p_total_reads_percell_zscored'] = np.log1p(adata_ESC_OCT4.obs['total_reads_percell_zscored'])
adata_ESC_OCT4.obs['log1p_non_zero_peaks_percell_zscored'] = np.log1p(adata_ESC_OCT4.obs['non_zero_peaks_percell_zscored'])


#adata_ESC_YAP1 = adata[(adata.obs['condition']=='ESC') & (adata.obs['TR']=='YAP1'),:].copy()

adata_EpiLC_MYC_all = adata[(adata.obs['condition']=='EpiLC') & (adata.obs['TR']=='MYC'),:].copy()
adata_EpiLC_MYC = adata_EpiLC_MYC_all[:, peaklist_Myc].copy()

# some calculations for the cells (after zscore filtering)
adata_EpiLC_MYC.obs['total_reads_percell_zscored'] = adata_EpiLC_MYC.X.sum(axis=1)
adata_EpiLC_MYC.obs['non_zero_peaks_percell_zscored'] = (adata_EpiLC_MYC.X != 0).sum(axis=1)

adata_EpiLC_MYC.obs['log1p_total_reads_percell_zscored'] = np.log1p(adata_EpiLC_MYC.obs['total_reads_percell_zscored'])
adata_EpiLC_MYC.obs['log1p_non_zero_peaks_percell_zscored'] = np.log1p(adata_EpiLC_MYC.obs['non_zero_peaks_percell_zscored'])


adata_EpiLC_NANOG_all = adata[(adata.obs['condition']=='EpiLC') & (adata.obs['TR']=='NANOG'),:].copy()
adata_EpiLC_NANOG = adata_EpiLC_NANOG_all[:, peaklist_Nanog].copy()

# some calculations for the cells (after zscore filtering)
adata_EpiLC_NANOG.obs['total_reads_percell_zscored'] = adata_EpiLC_NANOG.X.sum(axis=1)
adata_EpiLC_NANOG.obs['non_zero_peaks_percell_zscored'] = (adata_EpiLC_NANOG.X != 0).sum(axis=1)

adata_EpiLC_NANOG.obs['log1p_total_reads_percell_zscored'] = np.log1p(adata_EpiLC_NANOG.obs['total_reads_percell_zscored'])
adata_EpiLC_NANOG.obs['log1p_non_zero_peaks_percell_zscored'] = np.log1p(adata_EpiLC_NANOG.obs['non_zero_peaks_percell_zscored'])

adata_EpiLC_OCT4_all = adata[(adata.obs['condition']=='EpiLC') & (adata.obs['TR']=='OCT4'),:].copy()
adata_EpiLC_OCT4 = adata_EpiLC_OCT4_all[:, peaklist_Oct4].copy()

# some calculations for the cells (after zscore filtering)
adata_EpiLC_OCT4.obs['total_reads_percell_zscored'] = adata_EpiLC_OCT4.X.sum(axis=1)
adata_EpiLC_OCT4.obs['non_zero_peaks_percell_zscored'] = (adata_EpiLC_OCT4.X != 0).sum(axis=1)

adata_EpiLC_OCT4.obs['log1p_total_reads_percell_zscored'] = np.log1p(adata_EpiLC_OCT4.obs['total_reads_percell_zscored'])
adata_EpiLC_OCT4.obs['log1p_non_zero_peaks_percell_zscored'] = np.log1p(adata_EpiLC_OCT4.obs['non_zero_peaks_percell_zscored'])

#adata_EpiLC_YAP1 = adata[(adata.obs['condition']=='EpiLC') & (adata.obs['TR']=='YAP1'),:].copy()


print("MYC", adata_ESC_MYC_all.shape, adata_ESC_MYC.shape, adata_ESC_MYC.obs['FRiP'].isna().sum())
print("MYC", adata_EpiLC_MYC_all.shape, adata_EpiLC_MYC.shape, adata_EpiLC_MYC.obs['FRiP'].isna().sum())

print("NANOG", adata_ESC_NANOG_all.shape, adata_ESC_NANOG.shape, adata_ESC_NANOG.obs['FRiP'].isna().sum())
print("NANOG", adata_EpiLC_NANOG_all.shape, adata_EpiLC_NANOG.shape, adata_EpiLC_NANOG.obs['FRiP'].isna().sum())

print("OCT4", adata_EpiLC_MYC_all.shape, adata_ESC_OCT4.shape, adata_ESC_OCT4.obs['FRiP'].isna().sum())
print("OCT4", adata_EpiLC_OCT4_all.shape, adata_EpiLC_OCT4.shape, adata_EpiLC_OCT4.obs['FRiP'].isna().sum())

#print(adata_ESC_YAP1.shape)
#print(adata_EpiLC_YAP1.shape)




/var/folders/yv/0k5783m54kz5cltf2_077mrm0000gn/T/ipykernel_69932/3486663857.py:5: ImplicitModificationWarning: Setting element `.layers['normalized']` of view, initializing view as actual.
  adata.layers['normalized'] = adata.X.copy()
/Users/nadia/anaconda3/envs/scanpy/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:234: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


MYC (344, 24897) (344, 10048) 0
MYC (377, 24897) (377, 10048) 0
NANOG (295, 24897) (295, 10328) 0
NANOG (401, 24897) (401, 10328) 0
OCT4 (377, 24897) (312, 11041) 0
OCT4 (388, 24897) (388, 11041) 0


# Filter
the way it is done by github code



In [ ]:
##### violin plots for ESC and EpiLC #####
##### for each TR in each condition separatly filter out the first and last 2% percentiles #####

import anndata as ad

#
#sc.pl.violin(adata_ESC, keys="log1p_non_zero_peaks_percell", groupby='TR', rotation=90)

lpercentile = 15
hpercentile = 95

low_percentile = np.percentile(adata_ESC_MYC.obs['log1p_non_zero_peaks_percell'], lpercentile)
high_percentile = np.percentile(adata_ESC_MYC.obs['log1p_non_zero_peaks_percell'], hpercentile)
adata_ESC_MYC_filtered = adata_ESC_MYC[(adata_ESC_MYC.obs['log1p_non_zero_peaks_percell'] >= low_percentile) & (adata_ESC_MYC.obs['log1p_non_zero_peaks_percell'] <= high_percentile)].copy()
print("ESC_MYC :", adata_ESC_MYC.n_obs, adata_ESC_MYC_filtered.n_obs)

low_percentile = np.percentile(adata_ESC_NANOG.obs['log1p_non_zero_peaks_percell'], lpercentile)
high_percentile = np.percentile(adata_ESC_NANOG.obs['log1p_non_zero_peaks_percell'], hpercentile)
adata_ESC_NANOG_filtered = adata_ESC_NANOG[(adata_ESC_NANOG.obs['log1p_non_zero_peaks_percell'] >= low_percentile) & (adata_ESC_NANOG.obs['log1p_non_zero_peaks_percell'] <= high_percentile)].copy()
print("ESC_NANOG :", adata_ESC_NANOG.n_obs, adata_ESC_NANOG_filtered.n_obs)

low_percentile = np.percentile(adata_ESC_OCT4.obs['log1p_non_zero_peaks_percell'], lpercentile)
high_percentile = np.percentile(adata_ESC_OCT4.obs['log1p_non_zero_peaks_percell'], hpercentile)
adata_ESC_OCT4_filtered = adata_ESC_OCT4[(adata_ESC_OCT4.obs['log1p_non_zero_peaks_percell'] >= low_percentile) & (adata_ESC_OCT4.obs['log1p_non_zero_peaks_percell'] <= high_percentile)].copy()
print("ESC_OCT4 :", adata_ESC_OCT4.n_obs, adata_ESC_OCT4_filtered.n_obs)


# Concatenate AnnData objects along observations (rows, cells)
#adata_ESC_combined = ad.concat([adata_ESC_MYC_filtered, adata_ESC_NANOG_filtered, adata_ESC_OCT4_filtered, adata_ESC_YAP1_filtered], join="inner")
adata_ESC_combined = ad.concat([adata_ESC_MYC_filtered, adata_ESC_NANOG_filtered, adata_ESC_OCT4_filtered], join="inner") ##witthout YAP1

# Define a filename for saving the violin plots
#violin_pdf_filename = "violin_plots_ESC.pdf"
#
# Create a PDF file to store the violin plots
#with PdfPages(violin_pdf_filename) as pdf:
#    # Plot the violin plots
#    sc.pl.violin(adata_ESC_combined, keys="log1p_non_zero_peaks_percell", groupby='TR', rotation=90, xlabel='ESC', show=False)
#    # x ticks are rotated by 90 degrees
#    plt.xticks(rotation=0)
#    # Save the current page
#    plt.tight_layout()
#    pdf.savefig()  # Save the current figure
#    plt.close()  # Close the figure to free memory

#sc.pl.violin(adata_ESC_combined, keys="log1p_non_zero_peaks_percell", groupby='TR', rotation=90, xlabel='ESC')

#
#sc.pl.violin(adata_EpiLC, keys="log1p_non_zero_peaks_percell", groupby='TR', rotation=90)

low_percentile = np.percentile(adata_EpiLC_MYC.obs['log1p_non_zero_peaks_percell'], lpercentile)
high_percentile = np.percentile(adata_EpiLC_MYC.obs['log1p_non_zero_peaks_percell'], hpercentile)
adata_EpiLC_MYC_filtered = adata_EpiLC_MYC[(adata_EpiLC_MYC.obs['log1p_non_zero_peaks_percell'] >= low_percentile) & (adata_EpiLC_MYC.obs['log1p_non_zero_peaks_percell'] <= high_percentile)].copy()
print("EpiLC_MYC :", adata_EpiLC_MYC.n_obs, adata_EpiLC_MYC_filtered.n_obs)

low_percentile = np.percentile(adata_EpiLC_NANOG.obs['log1p_non_zero_peaks_percell'], lpercentile)
high_percentile = np.percentile(adata_EpiLC_NANOG.obs['log1p_non_zero_peaks_percell'], hpercentile)
adata_EpiLC_NANOG_filtered = adata_EpiLC_NANOG[(adata_EpiLC_NANOG.obs['log1p_non_zero_peaks_percell'] >= low_percentile) & (adata_EpiLC_NANOG.obs['log1p_non_zero_peaks_percell'] <= high_percentile)].copy()
print("EpiLC_NANOG :", adata_EpiLC_NANOG.n_obs, adata_EpiLC_NANOG_filtered.n_obs)

low_percentile = np.percentile(adata_EpiLC_OCT4.obs['log1p_non_zero_peaks_percell'], lpercentile)
high_percentile = np.percentile(adata_EpiLC_OCT4.obs['log1p_non_zero_peaks_percell'], hpercentile)
adata_EpiLC_OCT4_filtered = adata_EpiLC_OCT4[(adata_EpiLC_OCT4.obs['log1p_non_zero_peaks_percell'] >= low_percentile) & (adata_EpiLC_OCT4.obs['log1p_non_zero_peaks_percell'] <= high_percentile)].copy()
print("EpiLC_OCT4 :", adata_EpiLC_OCT4.n_obs, adata_EpiLC_OCT4_filtered.n_obs)


# Concatenate AnnData objects along observations (rows, cells)
#adata_EpiLC_combined = ad.concat([adata_EpiLC_MYC_filtered, adata_EpiLC_NANOG_filtered, adata_EpiLC_OCT4_filtered, adata_EpiLC_YAP1_filtered], join="inner")
adata_EpiLC_combined = ad.concat([adata_EpiLC_MYC_filtered, adata_EpiLC_NANOG_filtered, adata_EpiLC_OCT4_filtered], join="inner") ##without YAP1

# Define a filename for saving the violin plots
#violin_pdf_filename = "violin_plots_EpiLC.pdf"
#
#with PdfPages(violin_pdf_filename) as pdf:
#    # Plot the violin plots
#    sc.pl.violin(adata_EpiLC_combined, keys="log1p_non_zero_peaks_percell", groupby='TR', rotation=90, xlabel='EpiLC', show=False)
#    # x ticks are rotated by 90 degrees
#    plt.xticks(rotation=0)
#    # Save the current page
#    plt.tight_layout()
#    pdf.savefig()  # Save the current figure
#    plt.close()  # Close the figure to free memory



# Combine ESC and EpiLC
adata_combined = ad.concat([adata_ESC_combined, adata_EpiLC_combined], join="inner")

# FIX: Create a new column that combines 'TR' and 'Condition' for grouping
adata_combined.obs['TR_Condition'] = adata_combined.obs['TR'].astype(str) + "_" + adata_combined.obs['condition'].astype(str)

# Violin Plot with combined group
violin_pdf_filename = "VP_log1pNonZeroPeaksPerCell_noZscore_QuantileFiltered.pdf"
with PdfPages(violin_pdf_filename) as pdf:
    plt.figure(figsize=(12,6))
    sc.pl.violin(
        adata_combined,
        keys="log1p_non_zero_peaks_percell",
        groupby="TR_Condition",  # Use the new combined column
        rotation=90 , 
        show=False
    )  
    plt.xticks(rotation=90)
    plt.xticks(fontsize=8) # Reduce font size
    plt.yticks(fontsize=8) # Reduce font size
    plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels
    plt.title("log1p_non_zero_peaks_percell_nozscored_QuantileFiltered")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

# Violin Plot with combined group
violin_pdf_filename = "VP_log1pTotalReadsPercell_noZscore_QuantileFiltered.pdf"
with PdfPages(violin_pdf_filename) as pdf:
    plt.figure(figsize=(12,6))
    sc.pl.violin(
        adata_combined,
        keys="log1p_total_reads_percell",
        groupby="TR_Condition",  # Use the new combined column
        rotation=90 , 
        show=False
    )  
    plt.xticks(rotation=90)
    plt.xticks(fontsize=8) # Reduce font size
    plt.yticks(fontsize=8) # Reduce font size
    plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels
    plt.title("log1p_total_reads_percell_nozscored_QuantileFiltered")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

    print("adata_combined : ", adata_combined.shape)


# Violin Plot with combined group
violin_pdf_filename = "VP_log1pNonZeroPeaksPerCell_ZscoreFilterred_QuantileFiltered.pdf"
with PdfPages(violin_pdf_filename) as pdf:
    plt.figure(figsize=(12,6))
    sc.pl.violin(
        adata_combined,
        keys="log1p_non_zero_peaks_percell_zscored",
        groupby="TR_Condition",  # Use the new combined column
        rotation=90 , 
        show=False
    )  
    plt.xticks(rotation=90)
    plt.xticks(fontsize=8) # Reduce font size
    plt.yticks(fontsize=8) # Reduce font size
    plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels
    plt.title("log1p_non_zero_peaks_percell_zscored_QuantileFiltered")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

# Violin Plot with combined group
violin_pdf_filename = "VP_log1pTotalReadsPercell_ZscoreFilterred_QuantileFiltered.pdf"
with PdfPages(violin_pdf_filename) as pdf:
    plt.figure(figsize=(12,6))
    sc.pl.violin(
        adata_combined,
        keys="log1p_total_reads_percell_zscored",
        groupby="TR_Condition",  # Use the new combined column
        rotation=90 , 
        show=False
    )  
    plt.xticks(rotation=90)
    plt.xticks(fontsize=8) # Reduce font size
    plt.yticks(fontsize=8) # Reduce font size
    plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels
    plt.title("log1p_total_reads_percell_zscored_QuantileFiltered")
    plt.tight_layout()
    pdf.savefig()
    plt.close()

    print("adata_combined : ", adata_combined.shape)





ESC_MYC : 344 274
ESC_NANOG : 295 235
ESC_OCT4 : 312 250
EpiLC_MYC : 377 302
EpiLC_NANOG : 401 321
EpiLC_OCT4 : 388 313


/var/folders/yv/0k5783m54kz5cltf2_077mrm0000gn/T/ipykernel_69932/1856862370.py:131: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/var/folders/yv/0k5783m54kz5cltf2_077mrm0000gn/T/ipykernel_69932/1856862370.py:151: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


adata_combined :  (1695, 2018)
adata_combined :  (1695, 2018)


/var/folders/yv/0k5783m54kz5cltf2_077mrm0000gn/T/ipykernel_69932/1856862370.py:174: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
/var/folders/yv/0k5783m54kz5cltf2_077mrm0000gn/T/ipykernel_69932/1856862370.py:194: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

<Figure size 1200x600 with 0 Axes>

In [14]:
# Violin Plot with combined group FRiP
violin_pdf_filename = "VP_FRiP_noZscore_QuantileFiltered.pdf"
with PdfPages(violin_pdf_filename) as pdf:
    plt.figure(figsize=(12,6))
    sc.pl.violin(
        adata_combined,
        keys="FRiP",
        groupby="TR_Condition",  # Use the new combined column
        rotation=90 , 
        show=False
    )  
    plt.xticks(rotation=90)
    plt.xticks(fontsize=8) # Reduce font size
    plt.yticks(fontsize=8) # Reduce font size
    plt.xticks(rotation=45, ha="right")  # Rotate x-axis labels
    plt.tight_layout()
    pdf.savefig()
    plt.close()


/var/folders/yv/0k5783m54kz5cltf2_077mrm0000gn/T/ipykernel_69932/1909416127.py:16: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


<Figure size 1200x600 with 0 Axes>